In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset
from utils.harmonic_regressor import HarmonicRegressor
from utils.ee_utils import back_to_int, export_image_to_asset

from vegetation_period_NDVI.data_loading import add_time_data

from typing import List, Tuple

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

## Fill gaps in Landsat ET product using Harmonic Regression
### The units are mm/month NOT scaled. The gap filled collection will be scaled

In [5]:
start_date = ee.Date("2018-01-01")
end_date = ee.Date("2023-01-01")

landsat_data = ee.ImageCollection("projects/thurgau-irrigation/assets/ETlandsatmonthly")

landsat_data = landsat_data.filterDate(start_date, end_date)

PROJECTION = landsat_data.first().projection()
SCALE = PROJECTION.nominalScale()

# Prepare the bands for the harmonic regression
landsat_data = landsat_data.map(add_time_data)

landsat_data_list = landsat_data.toList(landsat_data.size())

In [6]:
regressor = HarmonicRegressor(vegetation_index="ET")

regressor.fit(landsat_data)
landsat_data_gap_filled = regressor.predict(landsat_data)

landsat_data_gap_filled = landsat_data_gap_filled.map(
    lambda img: back_to_int(
        img.select(["fitted"])
        .rename(f"fitted_ET")
        .setDefaultProjection(crs=PROJECTION, scale=SCALE),
        100,
    )
)

landsat_data_gap_filled_list = landsat_data_gap_filled.toList(
    landsat_data_gap_filled.size()
)

In [7]:
Map = geemap.Map()

image_first = ee.Image(landsat_data_list.get(7))
image_after = ee.Image(landsat_data_gap_filled_list.get(7))

vis_params = {
    "bands": ["ET"],
    "min": 0,
    "max": 200,
    "palette": ["blue", "yellow", "orange", "red"],
}

vis_params_fitted = {
    "bands": ["fitted_ET"],
    "min": 0,
    "max": 20000,
    "palette": ["blue", "yellow", "orange", "red"],
}

Map.center_object(image_first, 10)
Map.addLayer(image_first, vis_params, "ET")
Map.addLayer(image_after, vis_params_fitted, "ET fitted")

Map


Map(center=[46.81709734969232, 8.224625723540559], controls=(WidgetControl(options=['position', 'transparent_b…

## Export the gap filled Landsat ET collection. 
### The collection has now been scaled by 100!

In [8]:
def export_landsat_ET(
    landsat_et_collection: ee.ImageCollection,
    year: str,
    scale: float,
) -> List[ee.batch.Task]:
    """
    Export the Landsat ET data for a given year.

    Args:
        landsat_et_collection (ee.ImageCollection): The Landsat ET data collection.
        year (str): The year for which the data should be exported.
        scale (float): The scale of the exported images.


    Returns:
        List[ee.batch.Task]: A list of export tasks for the downscaled images.
    """
    landsat_et_collection_list = landsat_et_collection.toList(
        landsat_et_collection.size()
    )

    tasks = []
    for i in range(12):
        m = i + 1
        date = ee.Date.fromYMD(int(year), m, 1)
        time_step_name = f"{m:02d}"

        landsat_image = ee.Image(landsat_et_collection_list.get(i))

        # Change the date to the first day of the month
        landsat_image = landsat_image.set("system:time_start", date.millis())

        # Take the image's geometry as the AOI
        aoi = landsat_image.geometry()

        task_name = f"Landsat_ET_gap_filled_{year}-{time_step_name}"
        asset_id = f"projects/thurgau-irrigation/assets/Thurgau/projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/{task_name}"

        task = export_image_to_asset(landsat_image, asset_id, task_name, year, aoi)
        tasks.append(task)

    return tasks


years = range(2018, 2023)

for year in years:

    landsat_data_gap_filled_year = landsat_data_gap_filled.filterDate(
        ee.Date(f"{year}-01-01"), ee.Date(f"{year}-12-31")
    )

    tasks = export_landsat_ET(landsat_data_gap_filled_year, year, 30)

Exporting Landsat_ET_gap_filled_2018-01 for 2018 to projects/thurgau-irrigation/assets/Thurgau/projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/Landsat_ET_gap_filled_2018-01
Exporting Landsat_ET_gap_filled_2018-02 for 2018 to projects/thurgau-irrigation/assets/Thurgau/projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/Landsat_ET_gap_filled_2018-02
Exporting Landsat_ET_gap_filled_2018-03 for 2018 to projects/thurgau-irrigation/assets/Thurgau/projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/Landsat_ET_gap_filled_2018-03
Exporting Landsat_ET_gap_filled_2018-04 for 2018 to projects/thurgau-irrigation/assets/Thurgau/projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/Landsat_ET_gap_filled_2018-04
Exporting Landsat_ET_gap_filled_2018-05 for 2018 to projects/thurgau-irrigation/assets/Thurgau/projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/Landsat_ET_gap_filled_2018-05
